<a href="https://colab.research.google.com/github/RachelRFode/wilted/blob/master/wilted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Summary
Use shell and python
Get list of all of the tiles
Function takes one of those file names into device id, timestamp, hash (python)

Imports

In [0]:
import os
import pandas as pd
# using timestamp parsing function
from datetime import datetime
# sorting function
from operator import itemgetter, attrgetter
# time
from datetime import datetime, timedelta
# csv exporter
import csv

Authenticate google to obtain data

In [0]:
# Login to grab data from google cloud
project_id = 'grownome'
from google.colab import auth
auth.authenticate_user()

OPTIONAL

In [0]:
!gsutil -m rsync -r gs://grownome.appspot.com/images/ gs://grownome-vcm/images/

Building synchronization state...
At destination listing 10000...
Starting synchronization...
Copying gs://grownome.appspot.com/images/2538009072846140/1540492560642-27889e4d7a4a5c18fe63bbba80530005.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1540492860792-cb8a74bd3a3376e32a7d59ef931db42e.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2578132625949784/1540492666361-281bb2fc203121a28aa6483d66b9616a.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2662495380132089/1540492553123-2dc8f1221d37faeb983d3757afad1963.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2639491175006590/1540492545123-0df35df1c662dc32b8aad1aed00484e3.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2716136462675365/1540492738047-6ef9afc6360a1c02cba489f0de5b10eb.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2662495380132089/1540492853064-ab2c5b

Pull images from folder in Google Cloud

In [0]:
# Get data from google cloud  and assign it to list
!gsutil ls gs://grownome-vcm/images/2538009072846140/ > imagelist.txt
  
#check
#!cat imagelist.txt

Pull data

In [0]:
def parse_img_list(filepath):
  # create list for raw images
  raw_img_paths = []

  # get handle to txt file, read lines out of txt file, 
  # split by '\n', close handle
  with open(filepath) as f:
    raw_img_paths = f.read().split('\n')

  # create list for img paths
  parsed_img_paths = []

  # parsing img path filenames
  for img_path in raw_img_paths:
    if img_path != '':
      # split the url
      split_img_path = img_path.split('/')
      # get last device id and img filename from url
      device_id = split_img_path[4]
      file_name = split_img_path[5]
      # split img file name by '-'
      file_name_parts = file_name.split('-')
      
      if(len(file_name_parts[0]) == 13):
        # convert timestamp to int
        # divide timestamp by 1000 to convert from miliseconds to seconds
        timestamp = int(file_name_parts[0])/1000
        # parse timestamp into time object
        parsed_timestamp = datetime.utcfromtimestamp(timestamp)
        # putting device_id, timestamp, and img_path into dictionary
        parsed_part = {'device_id': device_id, 
                       'timestamp': parsed_timestamp,
                       'img_path': img_path}
        # append parsed part to list
        parsed_img_paths.append(parsed_part)

  # check: print list
  return parsed_img_paths

unsorted_img_list = parse_img_list('imagelist.txt')

Sort list by timestamp

In [0]:
sorted_img_list = sorted(unsorted_img_list, key=itemgetter('timestamp'))

Query all humidity readings for device

In [0]:
%%bigquery --project grownome humidity_query
# Grownome is the database and humidity_query is python variable
# that result is being assigned to.
SELECT
  # selecting the database columns
  # cast deviceNumId from numeric type to string type
  CAST(deviceNumId AS STRING) AS deviceId,
  humidity,
  timestamp
# metrics from grownome database from grownome project
FROM `grownome.grownome.metrics`
# with humidity reading for specific device
WHERE
  humidity IS NOT NULL AND 
  deviceNumId = 2538009072846140
ORDER BY
  timestamp

,deviceId,humidity,timestamp
0,2538009072846140,39.0,2018-09-22 18:14:00.632000+00:00
1,2538009072846140,38.0,2018-09-22 18:16:01.408000+00:00
2,2538009072846140,38.0,2018-09-22 18:18:03.157000+00:00
3,2538009072846140,38.0,2018-09-22 18:20:32.660000+00:00
4,2538009072846140,38.0,2018-09-22 18:21:44.303000+00:00
5,2538009072846140,38.0,2018-09-22 18:23:44.771000+00:00
6,2538009072846140,38.0,2018-09-22 18:25:45.255000+00:00
7,2538009072846140,38.0,2018-09-22 18:27:45.924000+00:00
8,2538009072846140,39.0,2018-09-22 18:29:46.570000+00:00
9,2538009072846140,39.0,2018-09-22 18:31:47.101000+00:00


Query Info

In [0]:
#humidity_query.describe()
humidity_series = humidity_query.set_index('timestamp')['humidity']

In [0]:
img_with_humidity = []

for item in sorted_img_list:
  label_value = 'default'
  # iloc() is integer lookup
  humidity_value = humidity_series[item['timestamp']-timedelta(hours=30):].iloc[0]
  item['humidity'] = humidity_value
  if humidity_value > 38:
    item['label'] = 'WATERED'
  else:
    item['label'] = 'DRY'
  img_with_humidity.append(item)
 
img_with_humidity

[{'device_id': '2538009072846140',
  'humidity': 41.0,
  'img_path': 'gs://grownome-vcm/images/2538009072846140/1537916992809-16152b799689c9d6d6eb81a669010679.jpg',
  'label': 'WATERED',
  'timestamp': datetime.datetime(2018, 9, 25, 23, 9, 52, 809000)},
 {'device_id': '2538009072846140',
  'humidity': 39.0,
  'img_path': 'gs://grownome-vcm/images/2538009072846140/1537935318736-63619e68e37d1fb46817d37cb682edaf.jpg',
  'label': 'WATERED',
  'timestamp': datetime.datetime(2018, 9, 26, 4, 15, 18, 736000)},
 {'device_id': '2538009072846140',
  'humidity': 38.0,
  'img_path': 'gs://grownome-vcm/images/2538009072846140/1538109292162-63957467727403b4ea34659d5d7959c4.jpg',
  'label': 'DRY',
  'timestamp': datetime.datetime(2018, 9, 28, 4, 34, 52, 162000)},
 {'device_id': '2538009072846140',
  'humidity': 37.0,
  'img_path': 'gs://grownome-vcm/images/2538009072846140/1538110519333-b247cc148423b2679f17273561d79500.jpg',
  'label': 'DRY',
  'timestamp': datetime.datetime(2018, 9, 28, 4, 55, 19, 33

In [0]:
img_with_humidity

with open('img_with_humidity.csv', 'w') as csvfile:
    fieldnames = ['img_path', 'label']
    
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)

    for item in img_with_humidity:
      
      writer.writerow(item)

In [0]:
!gsutil cp ./img_with_humidity.csv gs://grownome-vcm/files.csv

Copying file://./img_with_humidity.csv [Content-Type=text/csv]...
/ [1 files][306.7 KiB/306.7 KiB]                                                
Operation completed over 1 objects/306.7 KiB.                                    


Upload File to Google - AutoML Vision


*   make data set that assumes that plants with low humidity are wilted
*   label them all as low, med, high (add a label into each item of the list)
*  print into csv file
*  upload to google cloud (gsutil)

also try the humidity 3 hrs ago

you havd 7:00 timestamps - if humid is high and it is 7:00 am assume it is watered
